In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from skimage.transform import hough_line, hough_line_peaks
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.transform import resize
from skimage.feature import canny
from skimage.filters import sobel_h, sobel_v, gaussian
from skimage.draw import circle_perimeter, line
import skimage.io
import glob

In [ ]:
def vanishing_point(img, verbose=False):
    edges = np.zeros(img.shape[:2])
    for i in [0, 1, 2]:
        proc = img[:, :, i]
        proc = sobel_v(proc)
        edges += canny(proc, sigma=1)
    edges[5 * edges.shape[0] // 6:,:] = 0

    anglequery = np.linspace(-0.1, 0.1, 180)
    hspace, anglespace, distancespace = hough_line(edges, anglequery)

    accum, angles, dists = hough_line_peaks(
        hspace,
        anglespace,
        distancespace,
        threshold=0.8 * np.max(hspace)
    )

    if verbose:
        plt.figure()
        x0 = 0
        x1 = img.shape[1]
        for _, angle, dist in zip(accum, angles, dists):
            y0 = (dist - x0 * np.cos(angle)) / np.sin(angle)
            y1 = (dist - x1 * np.cos(angle)) / np.sin(angle)
#             plt.plot([x0, x1], [y0, y1], 'y-')
        plt.imshow(img)
        plt.xlim((0, img.shape[1]))
        plt.ylim((img.shape[0], 0))

    # https://en.wikipedia.org/wiki/Line–line_intersection#In_two_dimensions_2
    try:
        A = np.zeros((2, 2))
        B = np.zeros((2,1))
        for _, angle, dist in zip(accum, angles, dists):
            n = np.array([[np.cos(angle), np.sin(angle)]]).transpose()
            p = n * dist
            A += np.dot(n, n.transpose())
            B += np.dot(np.dot(n, n.transpose()), p)
        intersection = np.dot(np.linalg.inv(A), B)
    except np.linalg.LinAlgError:
        intersection = np.zeros((2, 1))
        
        
    for i in np.arange(0, img.shape[1], 50):
        rr, cc = line(int(intersection[1][0]), int(intersection[0][0]), img.shape[0], i)
        plt.plot(cc, rr, 'y,-')
    
    return intersection.reshape(2)

In [ ]:
def bullseye(img, verbose=False):
    edges = np.zeros(img.shape[:2])
    for i in [0, 1, 2]:
        proc = img[:, :, i]
        proc = sobel_h(proc)
        proc = gaussian(proc, sigma=1.5)
        edges += canny(proc, sigma=1.5)

    edges[edges.shape[0]//3:, :] = 0
    hough_radii = np.linspace(40, edges.shape[0]//8, 50)
    hough_res = hough_circle(edges, hough_radii)
    accums, cx, cy, radii = hough_circle_peaks(
        hough_res,
        hough_radii,
        total_num_peaks=1
    )
    
    if verbose:
        plt.figure()
        plt.imshow(img)
        plt.plot(cx, cy, 'yo')
        plt.plot(cx + radii, cy, 'y.')
        plt.plot(cx - radii, cy, 'y.')
        plt.plot(cx, cy + radii, 'y.')
        plt.plot(cx, cy - radii, 'y.')
    
    if len(accums) == 0:
        return np.array([0, 0, 0])
    else:
        return np.array([cx[0], cy[0], radii[0]])

In [ ]:
def process(img):    
    return np.concatenate([vanishing_point(img), bullseye(img)])

def create_db():
    imgs = glob.glob('imgs/*.*')
    import random
    with open('db3.txt', 'w') as f:
        for i, fname in enumerate(imgs):
            print(i, fname)
            img = skimage.io.imread(fname)
            out = process(img)
            print(fname, out[0], out[1], out[2], out[3], out[4], file=f)

def read_db():
    db = {}
    with open('db3.txt') as f:
        for line in f:
            name, vx, vy, cx, cy, r = line.split(' ')
            if float(r) == 0:
                continue
            db[name] = {
                'vx': float(vx),
                'vy': float(vy),
                'cx': float(cx),
                'cy': float(cy),
                'r': float(r)
            }
            data = db[name]
            factor = 50 / data['r']
            x_ = int(500 - data['cx'] * factor)
            y_ = int(200 - data['cy'] * factor)
            vx_ = int(x_ + data['vx'] * factor)
            vy_ = int(y_ + data['vy'] * factor)
            data['x_'] = x_
            data['y_'] = y_
            data['vx_'] = vx_
            data['vy_'] = vy_
    return db

In [ ]:
# create_db()
db = read_db()

In [ ]:
def composite(name, dest, verbose=False):
    data = db[name]
    img = skimage.io.imread(name)
    if data['r'] == 0:
        return
    
    factor = 50 / data['r']
    new_shape = tuple(map(int, (img.shape[0] * factor, img.shape[1] * factor)))
    scaled = resize(img,  new_shape)

    out = np.zeros((1000, 1000, 3))
    x = int(500 - data['cx'] * factor)
    y = int(200 - data['cy'] * factor)
    
    start_x = max(x, 0)
    start_y = max(y, 0)

    extent_x = min(1000, x + scaled.shape[1]) - x
    extent_y = min(1000, y + scaled.shape[0]) - y
    out[start_y:y + extent_y, start_x:x + extent_x, :] = scaled[start_y - y:extent_y, start_x - x:extent_x, :]
    
    if verbose:
        rr, cc = circle_perimeter(200, 500, 50)
        out[rr, cc, :] = 1

        vx = int(x + data['vx'] * factor)
        vy = int(y + data['vy'] * factor)
        for i in np.arange(0, 999, 100):
            rr, cc = line(vy, vx, 999, i)
            for r, c in zip(rr, cc):
                if r >= 0 and c >= 0 and r < 1000 and c < 1000:
                    out[r, c, 1] = 1
    
    plt.imsave(dest, out)

In [ ]:
from networkx import DiGraph
G = DiGraph()

keys = [k for k in db if db[k]['vy_'] < -200]

for name in keys:
    G.add_node(name)

for src in keys:
    dists = []
    for dst in keys:
        dist = (db[src]['vx_'] - db[dst]['vx_'])**2 + (db[src]['vy_'] - db[dst]['vy_'])**2
        if db[src]['vy_'] < db[dst]['vy_']:
            dists.append((dst, dist))
    dists.sort(key=lambda x: x[1])

    for dst, dist in dists[:2]:
        G.add_edge(src, dst, weight=(db[src]['vx_'] - db[dst]['vx_']) ** 2)
        plt.plot(
            [db[name]['vx_'] for name in [src, dst]],
            [max(min(db[name]['vy_'], 2000), -6000) for name in [src, dst]],
            'g-'
        )
    
    
plt.plot(
    [db[name]['vx_'] for name in db],
    [max(min(db[name]['vy_'], 2000), -6000) for name in db],
    'k,'
)

In [ ]:
from networkx import shortest_path
from networkx.algorithms.dag import dag_longest_path, topological_sort

p = dag_longest_path(
    G,
    weight='weight'
)
plt.plot(
    [db[name]['vx_'] for name in keys],
    [-max(min(db[name]['vy_'], 2000), -20000) for name in keys],
    'k,'
)
plt.plot(
    [db[name]['vx_'] for name in p],
    [-max(min(db[name]['vy_'], 2000), -20000) for name in p],
    'g.-'
)
plt.yscale('log')
plt.grid()

In [ ]:
import os
os.system('rm outs/*')
for i, name in enumerate(topological_sort(G)):
    if name not in p:
        continue
    print(i, name)
    composite(name, 'outs/%03d0.png' % i, verbose=True)